## emigration_country_born.tsv

- Change according to file

In [7]:
import pandas as pd
import numpy

url = "../../Data/emigration/emigration_country_born.tsv"
table_raw = pd.read_csv(url,sep='\t')

#Edit year range to corresponding in .tsv file
year_range = range(2008,2024,1)

file_save_path = "../../Data/emigration/emigration_country_born.csv"


- Important changes here:

In [3]:
# CHANGE HERE 
#create corresponding columns from first column
age = []
agedef = []
birth_country = []
sex = []
country = []

for index,row in table_raw.iterrows():
    title_split = (row[0].split(","))
    
    # add data to respective columns
    age_str = title_split[1][1:]
    
    if age_str.isnumeric():
        age.append(age_str)

    else:
        if title_split[1]=="Y_GE100":
            age.append(str(100))
        elif title_split[1]=="Y_LT1":
            age.append(str(0))
        else:
            age.append(title_split[1]) 

    agedef.append(title_split[2])
    birth_country.append(title_split[3])
    sex.append(title_split[5])
    country.append(title_split[6])
    
    
# add lists into table
table_raw["age"]= age
table_raw["agedef"] = agedef
table_raw["birth_country"] = birth_country
table_raw["sex"]=sex
table_raw["country"]=country

#drop the first column 
table_raw.drop(columns=["freq,age,agedef,c_birth,unit,sex,geo\TIME_PERIOD"],inplace=True)


<>:41: SyntaxWarning: invalid escape sequence '\T'
<>:41: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipykernel_41315/3561704600.py:41: SyntaxWarning: invalid escape sequence '\T'
  table_raw.drop(columns=["freq,age,agedef,c_birth,unit,sex,geo\TIME_PERIOD"],inplace=True)
/tmp/ipykernel_41315/3561704600.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  title_split = (row[0].split(","))


#### Reindexing, renaming columns, adding ISO3 codes


In [4]:
# import ISO3 codes 
country_codes = pd.read_csv("../../Data/country-region-codes.csv")

#Creating new column names for renaming
new_columns = {}
for column in table_raw.columns:
    if column.strip().isnumeric():
        new_columns[column] = int(column.strip())
    else:
        new_columns[column]=column
        
table = table_raw.rename(columns=new_columns)
table.rename(columns={"country":"alpha-2"},inplace=True) #Renaming country column to alpha-2 so it matches with country-region-codes

# Merging table on country codes. 
table = pd.merge(table,country_codes,on="alpha-2").drop(columns=["country-code","alpha-2"]).rename(columns={"name":"Country","alpha-3":"ISO3"})

#reordering
new_order = list(table.columns[-6:])+[i for i in year_range]# IMPORTANT to set -6 to number of non numeric columns

table = table[new_order]

#### Fixing values
Renaming : to NaN
fixing provisional data

In [5]:
replaced = table.copy()

#fix numeric values
for column in replaced.columns:
    replaced[column] = replaced[column].apply(str.strip)

replaced.replace(to_replace=":",value=numpy.nan,inplace=True)

#tags
tags = ["b","p","ep","e","be","bep","P"]

#I used Deepseek AI for this, it was quite complex
pattern = r'\b(?:' + '|'.join(tags) + r')\b'

# Apply the replacement to all columns (or specify specific columns)
df = replaced.apply(lambda x: x.str.replace(pattern, '', regex=True).str.strip())



#### Saving to file
- Uncomment to save file

In [ ]:
display(df.head())

#Uncomment to save file
# df.to_csv(file_save_path,index=False)

,age,agedef,birth_country,sex,Country,ISO3,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,TOTAL,COMPLET,AD,F,Austria,AUT,0,0,NaN,0,...,0,1,0,1,0,0,0,0,0,0
1,TOTAL,COMPLET,AD,F,Belgium,BEL,NaN,NaN,2,2,...,2,1,0,1,1,0,0,1,1,0
2,TOTAL,COMPLET,AD,F,Bulgaria,BGR,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,TOTAL,COMPLET,AD,F,Switzerland,CHE,NaN,NaN,NaN,0,...,1,4,4,1,3,4,3,NaN,NaN,NaN
4,TOTAL,COMPLET,AD,F,Czechia,CZE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
